In [ ]:
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json, joblib, sys, os
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from joblib import dump, load
from QiskitRegressor import QiskitRegressor
# Qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit import Parameter, ParameterVector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_ibm_runtime import Batch
from qiskit_ibm_runtime import exceptions

In [ ]:
def mitarai(quantumcircuit,num_wires,paramname='x'):
    # encoding as proposed by Mitarai et al.
    num_features = num_wires
    features = ParameterVector(paramname,num_features*2)
    for i in range(num_wires):
        feature_idx = i % num_features  # Calculate the feature index using modulo
        quantumcircuit.ry(np.arcsin(features[feature_idx * 2]), i)
        quantumcircuit.rz(np.arccos(features[feature_idx * 2 + 1] ** 2), i)


def double_angle(quantumcircuit, num_wires,paramname='x'):
    #  creates a circuit that encodes features into wires via angle encoding with an RY then RZ gate
    #  the features are encoded 1-1 onto the qubits
    #  if more wires are passed then features the remaining wires will be filled from the beginning of the feature list
    num_features = num_wires
    features = ParameterVector(paramname,num_features*2)
    for i in range(num_wires):
        feature_index = i % num_features
        quantumcircuit.ry(features[feature_index], i)
        quantumcircuit.rz(features[feature_index], i)

def entangle_cnot(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cnot gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cx(i, 0)
        else:
            quantumcircuit.cx(i, i+1)


def entangle_cz(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cz gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cz(i, 0)
        else:
            quantumcircuit.cz(i, i+1)


def HardwareEfficient(quantumcircuit,num_wires,paramname='theta'):
    parameters = ParameterVector(paramname,num_wires*3)
    for qubit in range(num_wires):
        quantumcircuit.rx(parameters[qubit * 3], qubit)  
        quantumcircuit.rz(parameters[qubit * 3 + 1], qubit)  
        quantumcircuit.rx(parameters[qubit * 3 + 2], qubit)  
    entangle_cnot(quantumcircuit,num_wires)





# def circuit(nqubits):
#     qc = QuantumCircuit(nqubits)
#     mitarai(qc,nqubits)
#     entangle_cz(qc,nqubits)
#     qc.barrier()
#     mitarai(qc,nqubits,paramname='x1')
#     entangle_cz(qc,nqubits)
#     qc.barrier()
#     HardwareEfficient(qc,nqubits)
#     qc.barrier()
#     return qc


def circuit(nqubits,RUD=1,AL=1):
    qc = QuantumCircuit(nqubits)
    for i in range(RUD):
        double_angle(qc,nqubits,paramname=f'x{i}')
        qc.barrier()
        for j in range(AL):        
            HardwareEfficient(qc,nqubits,paramname=f'theta{i}_{j}')
            qc.barrier()
    return qc




top=os.getcwd()
with open(os.path.join(top,'0.1_5_DDCC_train.bin'),'rb') as f:
    ddcc_train = joblib.load(f)

with open(os.path.join(top,'0.1_5_DDCC_test.bin'),'rb') as f:
    ddcc_test = joblib.load(f)

with open(os.path.join(top,'0.1_5_DDCC_scaler.bin'),'rb') as f:
    ddcc_scaler = joblib.load(f)

X_ddcc_train, y_ddcc_train = ddcc_train['X'],ddcc_train['y']
X_ddcc_test, y_ddcc_test = ddcc_test['X'],ddcc_test['y']

X_ddcc_train = X_ddcc_train.reshape(-1,64,5)
X_ddcc_test = X_ddcc_test.reshape(-1,64,5)
y_ddcc_train = y_ddcc_train.reshape(-1,64)
y_ddcc_test = y_ddcc_test.reshape(-1,64)


# X_train, y_train = X_ddcc_train, y_ddcc_train
# X_test, y_test = X_ddcc_test, y_ddcc_test
X_train, y_train = X_ddcc_train, y_ddcc_train
X_test, y_test = X_ddcc_test, y_ddcc_test

# X_train = [X_train[i:i+4] for i in range(0,len(X_train),4)]
# X_test = [X_test[i:i+4] for i in range(0,len(X_test),4)]
scaler = ddcc_scaler

# print(len(X_train),X_train[0].shape,X_train[-1].shape)
print(y_train.shape, y_test.shape)




num_qubits = 5
RUD = 1
AL = 5





shots = 1024 * 3




# optimization_level = int(optimization_level)
# shots = int(shots)
# resilience_level = int(resilience_level)




# 
qc = circuit(num_qubits,RUD,AL)

observables_labels = ''.join(['I']*(num_qubits-1))+"Z"


In [ ]:
done = ['ol0_rl0','ol1_rl0','ol2_rl0','ol3_rl0','ol0_rl1','ol1_rl1','ol2_rl1','ol3_rl1','ol0_rl2']

In [ ]:
results = {}
for d in done:

    optimization_level = int(d.split('_')[0].replace('ol',''))
    resilience_level = int(d.split('_')[1].replace('rl',''))

    df = pd.read_csv(os.path.join(d,'model_log.csv')).dropna()
    sns.lineplot(data=df,x='Iteration',y='Cost',label=d)
    plt.show()
    
    model = QiskitRegressor(qc,
                            num_qubits,
                            AL,
                            RUD,
                            'fake', #'fake',
                            observables_labels,
                            channel='ibm_quantum',
                            instance='pinq-quebec-hub/univ-toronto/default',
                            parameterpath = f'{d}/final_state_model.bin',
                            optimization_level = optimization_level,
                            resilience_level = resilience_level,
                            shots = shots,
                            iterations = 1,
                            verbose = True,
                            n_jobs = -4)
    
    y_train_pred = model.predict(X_train,iters='trainpred')
    y_test_pred = model.predict(X_test,iters='testpred')
    
    unscaled_y_train = ddcc_scaler.inverse_transform(y_train)
    unscaled_y_train_pred = ddcc_scaler.inverse_transform(y_train_pred)
    
    unscaled_y_test = ddcc_scaler.inverse_transform(y_test)
    unscaled_y_test_pred = ddcc_scaler.inverse_transform(y_test_pred)

    r2_train = r2_score(y_train.flatten(),y_train_pred.flatten())
    r2_test = r2_score(y_test.flatten(),y_test_pred.flatten())
    plt.scatter(y_train.flatten(),y_train_pred.flatten(),label="R$^{2}$="+f"{r2_train:.2f}")
    plt.plot(y_train.flatten(),y_train.flatten(),'k--')
    plt.scatter(y_test.flatten(),y_test_pred.flatten(),label="R$^{2}$="+f"{r2_test:.2f}")
    plt.legend()
    plt.show()    

    results[d] = {'R2_Train':r2_train,'R2_Test':r2_test}

In [ ]:
resultsdf = pd.DataFrame.from_dict(results).T

In [ ]:
resultsdf

In [ ]:
resultsdf[["Opt. Level", "Res. Level"]] = [[int(j.strip('orl')) for j in i.split('_')] for i in resultsdf.index]

In [ ]:
resultsdf.idxmax()

In [ ]:
resultsmelt = resultsdf.rename(columns = {"R2_Train":"Train","R2_Test":"Test"}).melt(id_vars = ['Opt. Level', 'Res. Level'],value_vars=['Train','Test'])

In [ ]:
g = sns.FacetGrid(resultsmelt,col='Opt. Level')
g.map_dataframe(sns.scatterplot, x='Res. Level', y='value', hue='variable',style='variable',palette=sns.color_palette('Paired',2),markers={"Train":'o',"Test":"^"},edgecolor='k')
g.add_legend()
g.set_ylabels("R$^{2}$")
g.set(ylim=(-1,1),xticks = [0,1,2])
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/fakequebec_0.1_resopt.png'),dpi=300,bbox_inches='tight')
plt.show()